In [1]:
#!pip install tensorflow-determinism

import os
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(1)
import random

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD,Adam,Adagrad,RMSprop
print(tf.__version__)

2.12.0


In [2]:
### WARN: Must have ###
# to make this notebook's output stable across runs
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)
   os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [3]:
# fashion MNIST 읽어 와서 신경망에 입력할 형태로 변환
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

4422102/4422102 [==============================] - 1s 0us/step


In [4]:
# 모델을 설계해주는 함수(모델을 나타내는 객체 model을 반환)
def run_model(x_train, y_train):
    reset_random_seeds()
    model=Sequential()
    model.add(Dense(128,activation='relu',input_shape=(784,)))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=256, epochs=20)
    return model

In [5]:
### Update: 22.04.01
acc = 0
models = []
n_of_models = 1
n = 0
models.append(run_model(x_train, y_train))
print('Model {}th accuracy: {}'.format(n, models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
acc =  models[n].evaluate(x_test,y_test,verbose=0)[1]*100

g_org_model_acc = acc
print('Baseline (average) accuracy: {}'.format(acc))

models[0].summary()

Epoch 1/20
235/235 [==============================] - 6s 4ms/step - loss: 0.6140 - accuracy: 0.7880
Epoch 2/20
235/235 [==============================] - 1s 4ms/step - loss: 0.4130 - accuracy: 0.8543
Epoch 3/20
235/235 [==============================] - 1s 4ms/step - loss: 0.3678 - accuracy: 0.8696
Epoch 4/20
235/235 [==============================] - 1s 4ms/step - loss: 0.3421 - accuracy: 0.8765
Epoch 5/20
235/235 [==============================] - 1s 4ms/step - loss: 0.3193 - accuracy: 0.8839
Epoch 6/20
235/235 [==============================] - 1s 6ms/step - loss: 0.3031 - accuracy: 0.8905
Epoch 7/20
235/235 [==============================] - 1s 5ms/step - loss: 0.2939 - accuracy: 0.8939
Epoch 8/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2802 - accuracy: 0.8969
Epoch 9/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2729 - accuracy: 0.8986
Epoch 10/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2652 - accuracy: 0.9027

In [45]:
###################################
# 여기부터 : 아래 함수를 수정하여 본인만의 새로운 모델을 만드시오
###################################
def run_proposed_model(x_train, y_train):  
    reset_random_seeds()  
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(784,)))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])
    
    # [※주의] 변경하면 안되는 옵션 : epochs, batch_size, _train, _text
    model.fit(x_train, y_train, batch_size=256, epochs=20)
    return model

In [46]:
### Update: 22.04.01

n_of_models = 1  # 실행할 때마다 정확도 변경하는지 확인하고 싶을 때 개수 증가시켜서 확인 가능
acc = 0
pro_models = []
if tf.test.is_gpu_available():
  for n in range (0, n_of_models):
    pro_models.append(run_proposed_model(x_train, y_train))
    acc += pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100 # verbose=1 when you want to see the progress
    print('Model {}th accuracy: {}'.format(n, 
                                           pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc /= n_of_models
else:
  n_of_models = 1
  n = 0
  pro_models.append(run_proposed_model(x_train, y_train))
  print('Model {}th accuracy: {}'.format(n, 
                                         pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100))
  acc =  pro_models[n].evaluate(x_test,y_test,verbose=0)[1]*100 # verbose=1 when you want to see the progress

g_pro_model_acc = acc
print('Proposed (average) accuracy: {}'.format(acc))

Epoch 1/20
235/235 [==============================] - 3s 4ms/step - loss: 0.5066 - accuracy: 0.8142
Epoch 2/20
235/235 [==============================] - 1s 4ms/step - loss: 0.3555 - accuracy: 0.8696
Epoch 3/20
235/235 [==============================] - 1s 4ms/step - loss: 0.3175 - accuracy: 0.8827
Epoch 4/20
235/235 [==============================] - 1s 6ms/step - loss: 0.2955 - accuracy: 0.8896
Epoch 5/20
235/235 [==============================] - 2s 7ms/step - loss: 0.2760 - accuracy: 0.8966
Epoch 6/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2590 - accuracy: 0.9026
Epoch 7/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2500 - accuracy: 0.9046
Epoch 8/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2338 - accuracy: 0.9114
Epoch 9/20
235/235 [==============================] - 1s 4ms/step - loss: 0.2283 - accuracy: 0.9125
Epoch 10/20
235/235 [==============================] - 1s 5ms/step - loss: 0.2165 - accuracy: 0.9179

In [47]:
### Update: 23.03.28
#################################
# 내가 개선한 모델의 정확률을 기존과 비교하며 출력
if g_pro_model_acc > (g_org_model_acc + 0.3):
    print('SUCCESS! Difference: {0:0.3f}'.format(
                        (g_pro_model_acc - g_org_model_acc)))
else:
    print('TRY DIFFERENTLY! Difference: {0:0.3f}'.format(
                        (g_pro_model_acc - g_org_model_acc)))
print('TF {0} under GPU {1}'.format(
    tf.__version__, tf.test.is_gpu_available()))
import sys
print("python version", sys.version)
#################################
# ★★★ capture the below prints and put them in the report ★★★

SUCCESS! Difference: 1.060
TF 2.12.0 under GPU True
python version 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
